In [ ]:
from openai import OpenAI
import os

os.environ['OPENAI_API_KEY']='YOUR APIKEY GOES HERE'

model_client=OpenAI()

response = model_client.embeddings.create(
    input="Machine learning is fun",
    model="text-embedding-3-small"
)
embedding_vector = response.data[0].embedding

In [ ]:
from pinecone import Pinecone
pc=Pinecone(api_key="YOUR APIKEY GOES HERE")

index_name="index-py"
if index_name not in pc.list_indexes().names():
    pc.create_index(name=index_name,
                dimension=1536,
                metric="cosine")


In [ ]:
index=pc.Index(index_name)
index.upsert([{
    "id": "doc1",
    "values": embedding_vector,
    "metadata": {"text": "Machine learning is fun"}
}])

query_embed=model_client.embeddings.create(
    input="Tell me about machine learning",
    model='text-embedding-3-small',).data[0].embedding

result = index.query(
    vector=query_embed,
    top_k=3,
    include_metadata=True)

print(result)